https://www.kaggle.com/competitions/w2024-kaggle-contest


In [ ]:
# Import Statements
import os
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV

In [ ]:
# Mounting Drive
from google.colab import drive
drive.mount('/content/drive')
folder_path = './drive/MyDrive/STAT441_GROUP_PROJECT/w2024-kaggle-contest'
os.chdir(folder_path)
X_train = pd.read_csv('X_train.csv')
X_test = pd.read_csv('X_test.csv')
y_train = pd.read_csv('y_train.csv')

Mounted at /content/drive


In [ ]:
X_train.head()

,id,year,fw_start,fw_end,country,c_abrv,v1,v2,v3,v4,...,v278b,v278c_r,v279a,v279b,v279c_r,v279d_r,v280,v281a,v281a_r,v282
0,0,2020,202011,202011,804,UA,2,1,2,2,...,45,14.45,15,40,15.40,55,1,uk,475,32
1,1,2018,201809,201811,688,RS,2,1,2,2,...,12,17.12,18,25,18.25,73,2,sr,397,3
2,2,2017,201706,201803,352,IS,2,2,1,1,...,58,15.58,13,20,13.20,-5,2,is,72,87
3,3,2021,202105,202110,428,LV,1,2,1,1,...,42,8.42,8,51,8.51,9,1,lv,256,5157
4,4,2018,201801,201805,40,AT,1,1,2,2,...,51,12.51,13,53,13.53,62,1,de,158,64


In [ ]:
# columns_with_nan = X_train.columns[X_train.isna().any()].tolist() Don't trees handle empty?
X_columns_to_drop = ['id']
y_columns_to_drop = ['id']
X_test.drop(X_columns_to_drop, inplace=True, axis=1)
y_train.drop(y_columns_to_drop, inplace=True, axis=1)

In [ ]:
# Creating a parameter grid
param_grid = {
    'boosting_type': ['gbdt'],
    'objective': ['multiclass'],
    'metric': ['multi_logloss'],         # any ideas , would choosing appropriate function help in better results?
    'num_leaves': [15, 31, 50],           # Number of leaves in one tree
    'learning_rate': [0.05, 0.1, 0.2],    # Learning rate
    'feature_fraction': [0.8, 0.9, 1.0],  # Percentage of features to consider for each tree
    'bagging_fraction': [0.8, 0.9, 1.0],  # Percentage of data to use for each boosting iteration
    'bagging_freq': [5],                  # Frequency for bagging
    'verbose': [1]
}

In [ ]:
# Removing the following columns c_abrv, v228b, v231b, v233b, v251b, v275b_N2, v275b_N1, v281a

X_train.drop(['c_abrv', 'v228b', 'v231b', 'v233b', 'v251b', 'v275b_N2', 'v275b_N1', 'v281a'], axis=1, inplace=True)
X_test.drop(['c_abrv', 'v228b', 'v231b', 'v233b', 'v251b', 'v275b_N2', 'v275b_N1', 'v281a'], axis=1, inplace=True)


In [ ]:
# Find parameters with lightgbm
model_lgb = lgb.LGBMClassifier()
grid_search = GridSearchCV(model_lgb, param_grid, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)
print(grid_search.best_params_)


Fitting 5 folds for each of 81 candidates, totalling 405 fits


In [ ]:
# Make prediction of X_train and find score

y_pred = model_lgb.predict(X_train)
score = model_lgb.score(X_train, y_train)
print('Prediction of X_train:', y_pred)
print('Score:', score)


[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
Prediction of X_train: [2 2 4 ... 4 3 3]
Score: 0.7364375


In [ ]:
# Finding all correlation for all columns with each other find the top 100
correlation_matrix = X_train.corr()
top_100_correlations = correlation_matrix.unstack().sort_values(ascending=False).drop_duplicates()
top_100_correlations = top_100_correlations[1:101]
top_100_correlations

v262_8cat     v262_EISCED       0.999998
v263_EISCED   v263_8cat         0.999997
v262_ISCED_2  v262_ISCED_2b     0.999986
v243_edulvlb  v243_edulvlb_2    0.999979
v263_ISCED_2  v263_ISCED_2b     0.999978
                                  ...   
v179_DK       v176_DK           0.974104
v181_DK       v177_DK           0.973999
v176_DK       v177_DK           0.973833
v181_DK       v179_DK           0.973674
v262_cs_GB1   v262_cs_GB2       0.973188
Length: 100, dtype: float64